# 1. Tile DTMs 

We have now our DTMs in UTM CRS, but they are too large to be pushed into the remote repository. This notebook will help us tile our DTMs so we will then be able to push them into our remote repository. Some of these functions were written with the help of `ChatGPT` and then implemented for our project. 

In [1]:
import os
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rasterio.plot
from osgeo import gdal
from rasterio import plot, mask, Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.windows import Window
import matplotlib.pyplot as plt
import rasterstats
import rioxarray
from matplotlib_scalebar.scalebar import ScaleBar

In [2]:
def tile_dem(input_dem, tile_size_x, tile_size_y, output_directory):
    ds = gdal.Open(input_dem)
    band = ds.GetRasterBand(1)
    cols = ds.RasterXSize
    rows = ds.RasterYSize
    transform = ds.GetGeoTransform()
    projection = ds.GetProjection()

    for i in range(0, rows, tile_size_y):
        for j in range(0, cols, tile_size_x):
            tile_name = os.path.join(output_directory, f"tile_{i}_{j}.tif")
            tile_cols = min(tile_size_x, cols - j)
            tile_rows = min(tile_size_y, rows - i)

            driver = gdal.GetDriverByName("GTiff")
            tile_ds = driver.Create(tile_name, tile_cols, tile_rows, 1, band.DataType)
            tile_ds.SetProjection(projection)
            tile_ds.SetGeoTransform((transform[0] + j * transform[1], transform[1], transform[2], 
                                     transform[3] + i * transform[5], transform[4], transform[5]))

            # Read the corresponding data from the input DEM and write to the tile
            tile_data = band.ReadAsArray(j, i, tile_cols, tile_rows)
            tile_ds.GetRasterBand(1).WriteArray(tile_data)

            tile_ds.FlushCache()  # Flush the cache to write to disk
            tile_ds = None

    ds = None

In [3]:
# Example usage:
demdir = ['/home/jovyan/nz-landslides/0-brute-raw-data/dem/2016_tiles',
          '/home/jovyan/nz-landslides/0-brute-raw-data/dem/2017_tiles',
          '/home/jovyan/nz-landslides/0-brute-raw-data/dem/2018_tiles']

dempath = ['/home/jovyan/nz-landslides_local/0-brute-raw-data/dem/2016_utm.tif',
           '/home/jovyan/nz-landslides_local/0-brute-raw-data/dem/2017_utm.tif',
           '/home/jovyan/nz-landslides_local/0-brute-raw-data/dem/2018_utm.tif']

for dir, path in zip(demdir, dempath): 
    
    input_dem = path  # Path to input DEM
    tile_size_x = 2048  # Tile size in X direction
    tile_size_y = 2048  # Tile size in Y direction
    output_directory = dir  # Output directory for tiles
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    # Call the function to tile the DEM
    tile_dem(input_dem, tile_size_x, tile_size_y, output_directory)
    
    print("Tiles creation completed: '{}'".format(dir))

/srv/conda/envs/notebook/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Tiles creation completed: '/home/jovyan/nz-landslides/0-brute-raw-data/dem/2016_tiles'
Tiles creation completed: '/home/jovyan/nz-landslides/0-brute-raw-data/dem/2017_tiles'
Tiles creation completed: '/home/jovyan/nz-landslides/0-brute-raw-data/dem/2018_tiles'
